In [1]:
import os
from delta import *
from delta.tables import *
import pyspark
from pyspark.sql import SparkSession


myAccessKey = '' 
mySecretKey = ''

def spark_session() -> SparkSession:
    os.environ['PYSPARK_SUBMIT_ARGS'] = f"""--packages io.delta:delta-core_2.12:2.1.0,org.apache.hadoop:hadoop-aws:3.3.1 
        --conf spark.hadoop.fs.s3a.access.key={myAccessKey} \
        --conf spark.hadoop.fs.s3a.secret.key={mySecretKey} pyspark-shell"""

    builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config('spark.sql.session.timeZone', 'UTC')

    spark = configure_spark_with_delta_pip(builder).getOrCreate()

    return spark


spark = spark_session()

23/05/21 10:16:32 WARN Utils: Your hostname, ProBook resolves to a loopback address: 127.0.1.1; using 172.22.34.28 instead (on interface eth0)
23/05/21 10:16:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/raddy/.pyenv/versions/3.10.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/raddy/.ivy2/cache
The jars for the packages stored in: /home/raddy/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2af07c2e-d509-4524-80d8-c4a7792f2acb;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.1!hadoop-aws.jar (111ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.901/aws-java-sdk-bundle

23/05/21 10:17:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/21 10:17:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.csv('./data/stocks.csv', header=True)

In [15]:
df.show()

+------+----------+-----+
|symbol|      date|price|
+------+----------+-----+
|  MSFT|Jan 1 2000|39.81|
|  MSFT|Feb 1 2000|36.35|
|  MSFT|Mar 1 2000|43.22|
|  MSFT|Apr 1 2000|28.37|
|  MSFT|May 1 2000|25.45|
|  MSFT|Jun 1 2000|32.54|
|  MSFT|Jul 1 2000| 28.4|
|  MSFT|Aug 1 2000| 28.4|
|  MSFT|Sep 1 2000|24.53|
|  MSFT|Oct 1 2000|28.02|
|  MSFT|Nov 1 2000|23.34|
|  MSFT|Dec 1 2000|17.65|
|  MSFT|Jan 1 2001|24.84|
|  MSFT|Feb 1 2001|   24|
|  MSFT|Mar 1 2001|22.25|
|  MSFT|Apr 1 2001|27.56|
|  MSFT|May 1 2001|28.14|
|  MSFT|Jun 1 2001| 29.7|
|  MSFT|Jul 1 2001|26.93|
|  MSFT|Aug 1 2001|23.21|
+------+----------+-----+
only showing top 20 rows



In [10]:
#df.write.format('delta').partitionBy('symbol').save('./data/delta/cars/')

In [8]:
spark.sql('SELECT * FROM delta.`/home/raddy/projects/DataLab/ml-area/data/delta/cars/` WHERE symbol="AMZN"').show()

+------+----------+-----+
|symbol|      date|price|
+------+----------+-----+
|  AMZN|Jan 1 2000|64.56|
|  AMZN|Feb 1 2000|68.87|
|  AMZN|Mar 1 2000|   67|
|  AMZN|Apr 1 2000|55.19|
|  AMZN|May 1 2000|48.31|
|  AMZN|Jun 1 2000|36.31|
|  AMZN|Jul 1 2000|30.12|
|  AMZN|Aug 1 2000| 41.5|
|  AMZN|Sep 1 2000|38.44|
|  AMZN|Oct 1 2000|36.62|
|  AMZN|Nov 1 2000|24.69|
|  AMZN|Dec 1 2000|15.56|
|  AMZN|Jan 1 2001|17.31|
|  AMZN|Feb 1 2001|10.19|
|  AMZN|Mar 1 2001|10.23|
|  AMZN|Apr 1 2001|15.78|
|  AMZN|May 1 2001|16.69|
|  AMZN|Jun 1 2001|14.15|
|  AMZN|Jul 1 2001|12.49|
|  AMZN|Aug 1 2001| 8.94|
+------+----------+-----+
only showing top 20 rows



In [2]:
sc = spark.sparkContext

In [3]:
cars = sc.textFile('./data/car-sales.csv')

In [9]:
cars = cars.map( lambda el: el.split(','))

In [10]:
cars_toyota = cars.filter(lambda zoneRow: zoneRow[0] == 'Toyota')

In [11]:
cars_toyota.take(5)

[['Toyota', 'White', '150043', '4', '"$4', '000.00"'],
 ['Toyota', 'Blue', '32549', '3', '"$7', '000.00"'],
 ['Toyota', 'Green', '99213', '4', '"$4', '500.00"'],
 ['Toyota', 'White', '60000', '4', '"$6', '250.00"']]

In [12]:
pair_rdd = cars.map(lambda car:(car[0],1))

In [13]:
pair_rdd.take(10)

[('Make', 1),
 ('Toyota', 1),
 ('Honda', 1),
 ('Toyota', 1),
 ('BMW', 1),
 ('Nissan', 1),
 ('Toyota', 1),
 ('Honda', 1),
 ('Honda', 1),
 ('Toyota', 1)]

In [14]:
pair_rdd = pair_rdd.reduceByKey(lambda x, y: x + y)

In [26]:
pair_rdd.sortBy(keyfunc=lambda x: x[1]).keys().take(4)

['Make', 'BMW', 'Nissan', 'Honda']